In [1]:
%%capture --no-stderr
%pip install -U langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph faiss-cpu

ERROR: Ignored the following versions that require a different python version: 0.0.10 Requires-Python >=3.9.0,<4.0; 0.0.11 Requires-Python >=3.9.0,<4.0; 0.0.12 Requires-Python >=3.9.0,<4.0; 0.0.13 Requires-Python >=3.9.0,<4.0; 0.0.14 Requires-Python >=3.9.0,<4.0; 0.0.15 Requires-Python >=3.9.0,<4.0; 0.0.16 Requires-Python >=3.9.0,<4.0; 0.0.17 Requires-Python >=3.9.0,<4.0; 0.0.18 Requires-Python >=3.9.0,<4.0; 0.0.19 Requires-Python >=3.9.0,<4.0; 0.0.20 Requires-Python >=3.9.0,<4.0; 0.0.21 Requires-Python >=3.9.0,<4.0; 0.0.22 Requires-Python >=3.9.0,<4.0; 0.0.23 Requires-Python >=3.9.0,<4.0; 0.0.24 Requires-Python >=3.9.0,<4.0; 0.0.25 Requires-Python >=3.9.0,<4.0; 0.0.26 Requires-Python >=3.9.0,<4.0; 0.0.27 Requires-Python >=3.9.0,<4.0; 0.0.28 Requires-Python >=3.9.0,<4.0; 0.0.9 Requires-Python >=3.9.0,<4.0; 0.1.1 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement langgraph (from versions: 0.0.8)
ERROR: No matching distribution found for langgraph


In [2]:
%pip install bs4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/expression_language/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# LCEL w/ PydanticOutputParser (outside the primary LCEL docs)
url = "https://python.langchain.com/docs/modules/model_io/output_parsers/quick_start"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
)
docs_pydantic = loader.load()

# LCEL w/ Self Query (outside the primary LCEL docs)
url = "https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
)
docs_sq = loader.load()

# Add
docs.extend([*docs_pydantic, *docs_sq])

# Sort the list based on the URLs in 'metadata' -> 'source'
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))

# Concatenate the 'page_content' of each sorted dictionary
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [6]:
from typing import Dict, TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [13]:
from operator import itemgetter

from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI


def generate(state: GraphState):
    """
    Generate a code solution based on LCEL docs and the input question
    with optional feedback from code execution tests

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """

    ## State
    state_dict = state["keys"]
    question = state_dict["question"]
    iter = state_dict["iterations"]

    ## Data model
    class code(BaseModel):
        """Code output"""

        prefix: str = Field(description="Description of the problem and approach")
        imports: str = Field(description="Code block import statements")
        code: str = Field(description="Code block not including import statements")

    ## LLM
    model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)

    # Tool
    code_tool_oai = convert_to_openai_tool(code)

    # LLM with tool and enforce invocation
    llm_with_tool = model.bind(
        tools=[code_tool_oai],
        tool_choice={"type": "function", "function": {"name": "code"}},
    )

    # Parser
    parser_tool = PydanticToolsParser(tools=[code])

    ## Prompt
    template = """You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
        Here is a full set of LCEL documentation: 
        \n ------- \n
        {context} 
        \n ------- \n
        Answer the user question based on the above provided documentation. \n
        Ensure any code you provide can be executed with all required imports and variables defined. \n
        Structure your answer with a description of the code solution. \n
        Then list the imports. And finally list the functioning code block. \n
        Here is the user question: \n --- --- --- \n {question}"""

    ## Generation
    if "error" in state_dict:
        print("---RE-GENERATE SOLUTION w/ ERROR FEEDBACK---")

        error = state_dict["error"]
        code_solution = state_dict["generation"]

        # Udpate prompt
        addendum = """  \n --- --- --- \n You previously tried to solve this problem. \n Here is your solution:  
                    \n --- --- --- \n {generation}  \n --- --- --- \n  Here is the resulting error from code 
                    execution:  \n --- --- --- \n {error}  \n --- --- --- \n Please re-try to answer this. 
                    Structure your answer with a description of the code solution. \n Then list the imports. 
                    And finally list the functioning code block. Structure your answer with a description of 
                    the code solution. \n Then list the imports. And finally list the functioning code block. 
                    \n Here is the user question: \n --- --- --- \n {question}"""
        template = template + addendum

        # Prompt
        prompt = PromptTemplate(
            template=template,
            input_variables=["context", "question", "generation", "error"],
        )

        # Chain
        chain = (
            {
                "context": lambda _: concatenated_content,
                "question": itemgetter("question"),
                "generation": itemgetter("generation"),
                "error": itemgetter("error"),
            }
            | prompt
            | llm_with_tool
            | parser_tool
        )

        code_solution = chain.invoke(
            {"question": question, "generation": str(code_solution[0]), "error": error}
        )

    else:
        print("---GENERATE SOLUTION---")

        # Prompt
        prompt = PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        )

        # Chain
        chain = (
            {
                "context": lambda _: concatenated_content,
                "question": itemgetter("question"),
            }
            | prompt
            | llm_with_tool
            | parser_tool
        )

        code_solution = chain.invoke({"question": question})

    iter = iter + 1
    return {
        "keys": {"generation": code_solution, "question": question, "iterations": iter}
    }


def check_code_imports(state: GraphState):
    """
    Check imports

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    ## State
    print("---CHECKING CODE IMPORTS---")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    imports = code_solution[0].imports
    iter = state_dict["iterations"]

    try:
        # Attempt to execute the imports
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        # Catch any error during execution (e.g., ImportError, SyntaxError)
        error = f"Execution error: {e}"
        if "error" in state_dict:
            error_prev_runs = state_dict["error"]
            error = error_prev_runs + "\n --- Most recent run error --- \n" + error
    else:
        print("---CODE IMPORT CHECK: SUCCESS---")
        # No errors occurred
        error = "None"

    return {
        "keys": {
            "generation": code_solution,
            "question": question,
            "error": error,
            "iterations": iter,
        }
    }


def check_code_execution(state: GraphState):
    """
    Check code block execution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    ## State
    print("---CHECKING CODE EXECUTION---")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    prefix = code_solution[0].prefix
    imports = code_solution[0].imports
    code = code_solution[0].code
    code_block = imports + "\n" + code
    iter = state_dict["iterations"]

    try:
        # Attempt to execute the code block
        exec(code_block)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        # Catch any error during execution (e.g., ImportError, SyntaxError)
        error = f"Execution error: {e}"
        if "error" in state_dict:
            error_prev_runs = state_dict["error"]
            error = error_prev_runs + "\n --- Most recent run error --- \n" + error
    else:
        print("---CODE BLOCK CHECK: SUCCESS---")
        # No errors occurred
        error = "None"

    return {
        "keys": {
            "generation": code_solution,
            "question": question,
            "error": error,
            "prefix": prefix,
            "imports": imports,
            "iterations": iter,
            "code": code,
        }
    }


### Edges


def decide_to_check_code_exec(state: GraphState):
    """
    Determines whether to test code execution, or re-try answer generation.

    Args:
       state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO TEST CODE EXECUTION---")
    state_dict = state["keys"]
    error = state_dict["error"]

    if error == "None":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TEST CODE EXECUTION---")
        return "check_code_execution"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: RE-TRY SOLUTION---")
        return "generate"


def decide_to_finish(state: GraphState):
    """
    Determines whether to finish (re-try code 3 times.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO TEST CODE EXECUTION---")
    state_dict = state["keys"]
    error = state_dict["error"]
    iter = state_dict["iterations"]

    if error == "None" or iter == 3:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TEST CODE EXECUTION---")
        return "end"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: RE-TRY SOLUTION---")
        return "generate"

In [14]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code_imports", check_code_imports)  # check imports
workflow.add_node("check_code_execution", check_code_execution)  # check execution

# Build graph
workflow.set_entry_point("generate")
workflow.add_edge("generate", "check_code_imports")
workflow.add_conditional_edges(
    "check_code_imports",
    decide_to_check_code_exec,
    {
        "check_code_execution": "check_code_execution",
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "check_code_execution",
    decide_to_finish,
    {
        "end": END,
        "generate": "generate",
    },
)

# Compile
app = workflow.compile()

In [17]:
import langsmith

client = langsmith.Client()

public_dataset = (
    "https://smith.langchain.com/public/326674a6-62bd-462d-88ae-eea49d503f9d/d"
)
# Clone the dataset to your tenant to use it
client.clone_public_dataset(public_dataset)

In [21]:
from langchain_core.runnables import RunnableLambda


## Data model
class code(BaseModel):
    """Code output"""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")


## LLM
model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)

# Tool
code_tool_oai = convert_to_openai_tool(code)

# LLM with tool and enforce invocation
llm_with_tool = model.bind(
    tools=[convert_to_openai_tool(code_tool_oai)],
    tool_choice={"type": "function", "function": {"name": "code"}},
)

# Parser
parser_tool = PydanticToolsParser(tools=[code])

# Create a prompt template with format instructions and the query
prompt = PromptTemplate(
    template="""You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
        Here is a full set of LCEL documentation: 
        \n ------- \n
        {context} 
        \n ------- \n
        Answer the user question based on the above provided documentation. \n
        Ensure any code you provide can be executed with all required imports and variables defined. \n
        Structure your answer with a description of the code solution. \n
        Then list the imports. And finally list the functioning code block. \n
        Here is the user question: \n --- --- --- \n {question}""",
    input_variables=["question", "context"],
)


def parse_answer_to_dict(x):
    return x[0].dict()


chain_base_case = (
    {
        "context": lambda _: concatenated_content,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm_with_tool
    | parser_tool
    | RunnableLambda(parse_answer_to_dict)
)

In [22]:
answer = chain_base_case.invoke("How can I write a RAG chain?")
answer

{'prefix': "To write a Retrieval-Augmented Generation (RAG) chain using LangChain Expression Language (LCEL), you can follow the steps outlined below. This example demonstrates how to create a RAG chain that incorporates a retrieval step to fetch relevant context before generating a response to a user's query. The chain consists of a retriever component to fetch relevant documents, a prompt template to format the query and context for the language model, and a language model to generate the response.\n\n",
 'imports': 'from operator import itemgetter\nfrom langchain_community.vectorstores import FAISS\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.runnables import RunnablePassthrough\nfrom langchain_openai import ChatOpenAI, OpenAIEmbeddings',
 'code': '# Initialize the vector store with sample texts and embeddings\nvectorstore = FAISS.from_texts(\n    ["harrison worked at kensho"], embedding=OpenAI

In [23]:
### No LangGraph

import uuid
from typing import Union

from langchain.smith import RunEvalConfig
from langsmith import Client
from langsmith.evaluation import EvaluationResult
from langsmith.schemas import Example, Run


def check_import(run: Run, example: Union[Example, None] = None):
    model_outputs = run.outputs
    imports = model_outputs["imports"]
    try:
        exec(imports)
        score = 1
    except:
        score = 0
    return EvaluationResult(key="check_import", score=score)


def check_execution(run: Run, example: Union[Example, None] = None):
    model_outputs = run.outputs
    imports = model_outputs["imports"]
    code = model_outputs["code"]
    code_to_execute = imports + "\n" + code
    try:
        exec(code_to_execute)
        score = 1
    except:
        score = 0
    return EvaluationResult(key="check_execution", score=score)


# Config
evaluation_config = RunEvalConfig(
    evaluators=[check_import, check_execution],
)

In [25]:
# Run eval on base chain
run_id = uuid.uuid4().hex[:4]
project_name = "context-stuffing-no-langgraph"
results = client.run_on_dataset(
    dataset_name="lcel-teacher-eval",
    llm_or_chain_factory=lambda: (lambda x: x["question"]) | chain_base_case,
    evaluation=evaluation_config,
    project_name=f"{run_id}-{project_name}",
)

View the evaluation results for project '2ec7-context-stuffing-no-langgraph' at:
https://smith.langchain.com/o/251ecdd9-2ca8-5dd9-ab04-c02e1fdaa47b/datasets/35eb6da5-2be1-4d79-b992-c8c8032be8ce/compare?selectedSessions=53708011-3fa4-4480-97a9-ed998714a35b

View all tests for Dataset lcel-teacher-eval at:
https://smith.langchain.com/o/251ecdd9-2ca8-5dd9-ab04-c02e1fdaa47b/datasets/35eb6da5-2be1-4d79-b992-c8c8032be8ce
[>                                                 ] 0/20content="Why don't bears wear socks?\n\nBecause they have bear feet!" response_metadata={'finish_reason': 'stop', 'logprobs': None}
[->                                                ] 1/20{'joke': "Why don't bears like fast food?\n\nBecause they can't catch it!", 'poem': "In the heart of the forest, wild and free\nRoams a creature, strong and mighty\nWith fur as dark as the night\nAnd eyes that gleam with pure delight\n\nThe bear, a symbol of power and grace\nNavigating the woods with steady pace\nA gentle giant, yet 

Chain failed for example 9fed0144-cd6b-4c61-918e-b88fd226167a with inputs {'question': 'How can I use a custom function to route between 2 chains in LCEL?'}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-kv3VikNMB4MVTBLwKd3vFvUk on tokens per min (TPM): Limit 300000, Used 289437, Requested 74889. Please try again in 12.865s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


[----------->                                      ] 5/20{'title': 'PromptInput', 'type': 'object', 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}
[----------------->                                ] 7/20Why did the bear break up with his girlfriend? 

Because he couldn't bear the relationship any longer!
{'history': [HumanMessage(content='hi im bob'), AIMessage(content='Hello Bob! How can I assist you today?')]}
[--------------------->                            ] 9/20

Chain failed for example 37b94cd3-d58a-4b9b-bbc1-09d3d8f8fec9 with inputs {'question': "My LCEL map contains the key 'question'. What is the difference between using itemgetter('question'), lambda x: x['question'], and x.get('question')?"}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-kv3VikNMB4MVTBLwKd3vFvUk on tokens per min (TPM): Limit 300000, Used 283927, Requested 74909. Please try again in 11.767s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


[------------------------>                         ] 10/20

...

[----------------------------->                    ] 12/20

Chain failed for example 577aa8c7-cbe7-49fe-bfa2-c3ceb48fbb68 with inputs {'question': "I'm invoking a LCEL chain with a map that contain {'question': 'how do I use Anthropic?'}. The full chain definition is full_chain = {'question': lambda x: x['question']} | sub_chain. Why is a lambda used?"}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-kv3VikNMB4MVTBLwKd3vFvUk on tokens per min (TPM): Limit 300000, Used 291978, Requested 74923. Please try again in 13.38s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


[---------------------------------->               ] 14/20

Chain failed for example ceefc5fe-b782-46a7-9bf5-d95d0d7fcd40 with inputs {'question': "I am passing a map with {'num': 1} to a LCEL chain. How can I add an extra key num2 to this map that adds 1 to the value of num. Then I want to assign this new map to a new key named output?"}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-kv3VikNMB4MVTBLwKd3vFvUk on tokens per min (TPM): Limit 300000, Used 286975, Requested 74920. Please try again in 12.379s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


[------------------------------------->            ] 15/20
[----------------------------------------->        ] 17/20content='Why do bears have hairy coats?\n\nFur protection!' response_metadata={'finish_reason': 'stop', 'logprobs': None}
[-------------------------------------------->     ] 18/20Harrison worked at Kensho.
[----------------------------------------------->  ] 19/20{'a': 1, 'b': 2, 'c': 3}
[------------------------------------------------->] 20/20

In [26]:
### LangGraph


def check_import(run: Run, example: Union[Example, None] = None):
    model_outputs = run.outputs["keys"]
    imports = model_outputs["imports"]
    try:
        exec(imports)
        score = 1
    except:
        score = 0
    return EvaluationResult(key="check_import", score=score)


def check_execution(run: Run, example: Union[Example, None] = None):
    model_outputs = run.outputs["keys"]
    imports = model_outputs["imports"]
    code = model_outputs["code"]
    code_to_execute = imports + "\n" + code
    try:
        exec(code_to_execute)
        score = 1
    except:
        score = 0
    return EvaluationResult(key="check_execution", score=score)


# Config
evaluation_config = RunEvalConfig(
    custom_evaluators=[check_import, check_execution],
)

config = {"recursion_limit": 50}


def model(input: dict):
    return app.invoke({"keys": {**input, "iterations": 0}}, config=config)


run_id = uuid.uuid4().hex[:4]
project_name = "context-stuffing-with-langgraph"
langgraph_results = client.run_on_dataset(
    dataset_name="lcel-teacher-eval",
    llm_or_chain_factory=model,
    evaluation=evaluation_config,
    project_name=f"{run_id}-{project_name}",
)

View the evaluation results for project '57ac-context-stuffing-with-langgraph' at:
https://smith.langchain.com/o/251ecdd9-2ca8-5dd9-ab04-c02e1fdaa47b/datasets/35eb6da5-2be1-4d79-b992-c8c8032be8ce/compare?selectedSessions=bdb57f76-effd-4fe9-bd15-0da4aefb4c65

View all tests for Dataset lcel-teacher-eval at:
https://smith.langchain.com/o/251ecdd9-2ca8-5dd9-ab04-c02e1fdaa47b/datasets/35eb6da5-2be1-4d79-b992-c8c8032be8ce
[>                                                 ] 0/20---GENERATE SOLUTION---
---GENERATE SOLUTION---
---GENERATE SOLUTION---
---GENERATE SOLUTION---
---GENERATE SOLUTION---
---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
---CHECKING CODE EXECUTION---
{'joke': "Why don't bears wear shoes?\nBecause they have bear feet!", 'poem': "In the forest deep and wild,\nWhere shadows dance and whispers mild,\nThe mighty bears roam free and strong,\nTheir fur a coat both thick and long.\n\nWith paws

Chain failed for example 9fed0144-cd6b-4c61-918e-b88fd226167a with inputs {'question': 'How can I use a custom function to route between 2 chains in LCEL?'}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-kv3VikNMB4MVTBLwKd3vFvUk on tokens per min (TPM): Limit 300000, Used 296682, Requested 74889. Please try again in 14.314s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


[--------->                                        ] 4/20---GENERATE SOLUTION---
---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
---CHECKING CODE EXECUTION---
Python is a versatile and powerful programming language that is known for its simplicity and readability. It was created by Guido van Rossum in the late 1980s and has since become one of the most popular programming languages in the world. One interesting fact about Python is that it is named after the British comedy show "Monty Python's Flying Circus," which Guido van Rossum was a fan of. Additionally, Python has a large and active community of developers who contribute to its open-source ecosystem, making it a great choice for beginners and experienced programmers alike.
---CODE BLOCK CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---


Chain failed for example 8ac4ac90-876a-4034-8ae0-6f5b77c5d8e7 with inputs {'question': 'How can I make the output of my LCEL chain a string?'}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-kv3VikNMB4MVTBLwKd3vFvUk on tokens per min (TPM): Limit 300000, Used 290110, Requested 74885. Please try again in 12.999s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


[----------->                                      ] 5/20---GENERATE SOLUTION---
---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
---CHECKING CODE EXECUTION---
Python programming language was created by Guido van Rossum and released in 1991. The name "Python" was inspired by the British comedy show "Monty Python's Flying Circus." Python is known for its simplicity and readability, making it a popular choice for beginners and experienced programmers alike. Python has a wide range of applications, including web development, data analysis, artificial intelligence, and automation. It has a large and active community of developers who contribute to its libraries, frameworks, and tools. Python is also the fastest-growing programming language according to the TIOBE Index, showing its increasing popularity and relevance in the tech industry.
---CODE BLOCK CHECK: FAILED---
---DECIDE TO TEST CODE EXECUTION---
---

...

[------------------------------->                  ] 13/20---GENERATE SOLUTION---
---GENERATE SOLUTION---


Chain failed for example ceefc5fe-b782-46a7-9bf5-d95d0d7fcd40 with inputs {'question': "I am passing a map with {'num': 1} to a LCEL chain. How can I add an extra key num2 to this map that adds 1 to the value of num. Then I want to assign this new map to a new key named output?"}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-kv3VikNMB4MVTBLwKd3vFvUk on tokens per min (TPM): Limit 300000, Used 296231, Requested 74920. Please try again in 14.23s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Chain failed for example c05120e1-7f1b-40d4-9a94-dd52d7610ee5 with inputs {'question': "I am passing text key 'foo' to my prompt and want to process it with a function, process_text(...), prior to the prompt. How can I do this using LCEL?"}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'R

[------------------------------------->            ] 15/20---GENERATE SOLUTION---
---GENERATE SOLUTION---


Chain failed for example 0d903a3e-91bc-4131-9d48-789b48eadf17 with inputs {'question': 'How can I configure the temperature of an LLM when invoking the LCEL chain?'}
Error Type: RateLimitError, Message: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-kv3VikNMB4MVTBLwKd3vFvUk on tokens per min (TPM): Limit 300000, Used 296151, Requested 74891. Please try again in 14.208s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


[--------------------------------------->          ] 16/20---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
---CHECKING CODE EXECUTION---
---CODE BLOCK CHECK: FAILED---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
[----------------------------------------->        ] 17/20---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
---CHECKING CODE EXECUTION---
{'a': 1, 'b': 2, 'c': 3}
---CODE BLOCK CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
{'a': 1, 'b': 2, 'c': 3}
---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
---CHECKING CODE EXECUTION---
---CODE BLOCK CHECK: FAILED---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: RE-TRY SOLUTION---
[--------------------------------------

In [27]:
# You will have to update these to match the tests you ran.
# The test name can be found at langgraph_results["project_name"]
langgraph = [
    "80db-context-stuffing-with-langgraph",
    "060c-context-stuffing-with-langgraph",
    "93cd-context-stuffing-with-langgraph",
    "60ef-context-stuffing-with-langgraph",
]

In [28]:
no_langgraph = [
    "b493-context-stuffing-no-langgraph",
    "eb8a-context-stuffing-no-langgraph",
    "b88c-context-stuffing-no-langgraph",
    "0aaa-context-stuffing-no-langgraph",
]

In [34]:
import pandas as pd


def prepare_dataframe(project, trial_number, chain):
    df = client.get_test_results(project_name=project)
    df = df.dropna(subset=["feedback.check_execution", "feedback.check_import"])
    df = df[["input.question", "feedback.check_execution", "feedback.check_import"]]
    df["trial #"] = trial_number
    df["chain"] = chain
    return df


# Prepare each dataframe
dfs_chain1 = [
    prepare_dataframe(project, i + 1, "LangGraph")
    for i, project in enumerate(langgraph)
]
dfs_chain2 = [
    prepare_dataframe(project, i + 1, "No LangGraph")
    for i, project in enumerate(no_langgraph)
]

# Combine all dataframes
final_df = pd.concat(dfs_chain1 + dfs_chain2, ignore_index=True)

LangSmithNotFoundError: Project 80db-context-stuffing-with-langgraph not found

In [ ]:
#### NO MORE MONEY